In [1]:
import os
import numpy as np
import scipy.io.wavfile as wf
import matplotlib.pyplot as plt
import librosa
import pandas as pd
from sklearn.metrics import classification_report

In [2]:
#load validation dataset
audio_dataset_validation = 'C:/Users/heera/Desktop/Dissertation/validation'

In [3]:
metadata_val=pd.read_csv('C:/Users/heera/Desktop/Dissertation/validation/REFERENCE.csv',header= None)

In [4]:
metadata_val.head()

,0,1,2
0,a0001.wav,a0001,1
1,a0002.wav,a0002,1
2,a0003.wav,a0003,1
3,a0004.wav,a0004,1
4,a0005.wav,a0005,1


In [5]:
metadata_val = metadata_val.rename(columns={0: "File_name",1:"names", 2: "Class"})

In [6]:
metadata_val['Class'] = metadata_val['Class'].map({1: 1, -1: 0})

In [7]:
metadata_val.head()

,File_name,names,Class
0,a0001.wav,a0001,1
1,a0002.wav,a0002,1
2,a0003.wav,a0003,1
3,a0004.wav,a0004,1
4,a0005.wav,a0005,1


In [8]:
metadata_val['Class'].value_counts()

1    151
0    150
Name: Class, dtype: int64

In [9]:
#create function to extract mfcc
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40,dct_type=2)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features
    

In [11]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features_val=[]
for index_num,row in tqdm(metadata_val.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_validation),str(row["File_name"]))
    final_class_labels=row["Class"]
    data=features_extractor(file_name)
    extracted_features_val.append([data,final_class_labels])

301it [00:51,  5.87it/s]


In [12]:
### converting extracted_features to Pandas dataframe
extracted_features_df_val=pd.DataFrame(extracted_features_val,columns=['feature','class'])
extracted_features_df_val.head()

,feature,class
0,"[-577.74457, 116.68117, 82.45421, 40.931873, 7...",1
1,"[-628.8163, 136.51126, 100.357864, 56.20959, 1...",1
2,"[-601.1834, 77.22223, 61.623146, 42.22163, 25....",1
3,"[-562.589, 88.77356, 71.55774, 49.522438, 29.5...",1
4,"[-489.47345, 65.72582, 57.030285, 45.383488, 3...",1


In [13]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df_val['feature'].tolist())
y=np.array(extracted_features_df_val['class'].tolist())

In [14]:
s = pd.DataFrame(data = X)

In [15]:
t = pd.DataFrame(data = y)

In [16]:
merged_val = pd.concat([s, t], axis=1)

In [17]:
merged_val.head(10)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,0
0,-577.744568,116.681168,82.454208,40.931873,7.485605,-7.887007,-4.491601,10.008358,24.508472,30.438593,...,5.994569,5.839571,3.671602,0.848826,-1.004319,-1.004396,0.530715,2.398161,3.331668,1
1,-628.816284,136.511261,100.357864,56.209591,19.551243,0.428206,-0.259094,10.158183,21.104572,24.655876,...,3.507886,2.841200,1.227278,-0.146953,-0.405241,0.449801,1.629177,2.186589,1.692338,1
2,-601.183411,77.222229,61.623146,42.221630,25.632828,16.337782,15.002048,18.738729,23.003593,24.039801,...,2.748103,2.922751,2.290715,1.166961,0.225971,0.042497,0.699116,1.713670,2.332496,1
3,-562.588989,88.773560,71.557739,49.522438,29.541950,16.664858,12.350879,14.427670,18.631382,20.879950,...,0.984788,1.662023,2.113323,2.125078,1.752352,1.235468,0.835051,0.692907,0.788556,1
4,-489.473450,65.725822,57.030285,45.383488,33.820339,24.795256,19.436211,17.373821,17.174334,17.152473,...,2.122123,2.094586,2.164926,2.183312,2.057845,1.794090,1.474256,1.199110,1.032410,1
5,-521.912231,109.789894,83.131897,51.075043,25.148706,12.496006,13.276185,21.479649,28.873482,29.626736,...,3.874432,3.368307,1.625785,-0.220157,-1.024072,-0.356238,1.263884,2.738425,3.091081,1
6,-498.956665,72.541809,63.870949,52.172474,40.260586,30.402330,23.705975,19.943716,17.949722,16.366316,...,1.218547,1.346841,1.449242,1.451255,1.343598,1.173696,1.008720,0.897983,0.852941,0
7,-608.290039,121.804886,83.961304,39.375343,5.452353,-7.348024,0.268497,18.132322,33.093048,36.229198,...,3.747848,3.705591,2.851964,2.033481,1.847434,2.232020,2.619894,2.462989,1.681644,1
8,-637.361816,136.915863,91.050957,37.788876,-1.203433,-13.863134,-2.607932,18.966412,34.974941,36.172955,...,3.691612,2.384445,0.977667,0.475651,0.934388,1.584123,1.558784,0.596325,-0.809997,0
9,-618.426880,119.006699,87.037468,48.069096,16.157154,0.287781,0.918467,11.101500,21.160604,24.063671,...,3.019052,2.170335,0.621219,-0.567352,-0.703185,0.121662,1.192087,1.730130,1.425894,1


In [20]:
merged_val.shape

(301, 41)

In [21]:
merged_val.to_csv('C:/Users/heera/Desktop/Dissertation/validation/merged_val.csv',index=False)